순환신경망 (Recurrent Neural Network, RNN)
RNN은 입력과 출력을 시퀀스 단위로 처리하는 시퀀스 모델이다. 가장 기본적인 시퀀스 인공신경망이고 LSTM이나 GRU또한 근본적으로 RNN에 속한다. 
기본적인 신경망 대부분은 은닉층(hidden layer)에서 출력층 방향으로만 향한다.
하지만 RNN의 경우 은닉층의 노드에서 나온 결과값을 출력층 방향으로 보내면서 다시 은닉층 노드의 다음 계산 입력으로 보내는 특징을 가지고 있다. 
RNN에서 은닉층의 결과를 내보내는 역할을 셀(cell)이라고 한다. 이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 메모리셀 RNN셀 이라고 표현한다.

은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점의 은닉층 메모리 셀에서 나온 값을 자신의 입력으로 사용한다. 
즉, 여러개의 입력데이터가 있고 여러개의 셀이 있다. 하나의 입력은 하나의 셀에 들어가서 예측값을 만들고 다음 시점의 셀로도 들어간다.

RNN은 입력과 출력의 길이를 다르게 설계할 수 있어서 다양한 용도로 사용할 수 있다. 

이렇게 되면 weight matrix가 총 3종류가 필요하다.
입력 단어벡터에 곱해지는 matrix 1, 이전 시점의 hidden state에 곱해지는 matrix 2, 처리된 이전 시점의 값과 현재 시점의 값을 더해서 생성된 hidden state에 곱해서 예측값을 만드는 matrix 3
이 matrix 3종류는 같은 layer에서 값을 공유한다. 또한 활성화함수는 하이퍼볼릭 탄젠트를 많이 사용한다.

코드는
from tensorflow.keras.layers import SimpleRNN

model.add(SimpleRNN(hidden_units,batch_input_shape=(batch_size,timesteps,input_dim)))
와 같이 사용할 수 있는데
hidden_units 는 은닉 상태의 크기를 정의한다. 메모리 셀이 다음시점으로 보내는 값의 크기이다. RNN의 용량(capacity)을 늘린다고 보면되고 중소형 모델의 경우 128, 256, 512, 1024 등의 값을 가진다.
timesteps 는 입력 시퀀스의 길이(input_length)이다.
input_dim 은 입력의 크기이다.

또한 return_sequences = T/F 인자를 통해서 False면 마지막 시점의 은닉상태만 출력하고 True면 모든 시점의 은닉상태를 출력한다. False를 통해서 다대일 문제를 해결할 수 있고 True를 통해 다음 은닉층이 하나 더 있는 경우나 다대다 문제를 해결할 수 있다.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3,batch_input_shape=(8,2,10)))
model.summary()
#output shape가 8,3으로 설정된 것을 볼 수 있다. (batch,outdim)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


양방향 순환 신경망 (Bidirectional Recurrent Neural Network)
양방향 순환 신경망은 이전 시점의 입력 뿐만 아니라 이후 시점의 입력또한 받아야된다는 아이디어에서 시작되었다.

양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 2개의 메모리 셀을 사용한다.
첫번째 메모리셀은 앞시점의 은닉상태를 다음시점으로 넘겨주기 위해서 존재한다. 두번째 메모리셀은 다음시점의 은닉상태를 전달받아서 현재의 은닉상태를 계산한다. 이 두개의 값 모두가 현재 시점의 출력층에서 출력값을 예측하기 위해서 사용된다.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional

time_steps = 10
input_dim = 5
model = Sequential()
model.add(Bidirectional(SimpleRNN(3,return_sequences=True),input_shape=(time_steps,input_dim)))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 10, 6)            54        
 l)                                                              
                                                                 
Total params: 54
Trainable params: 54
Non-trainable params: 0
_________________________________________________________________


위에서 살펴본 RNN을 가장 기본적인 RNN모델이라고 해서 Vanilla RNN이라고 부른다. (keras에서는 SimpleRNN)
바닐라 RNN의 한계는 시점이 많아질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 있다. 이를 장기 의존성 문제라고 한다 (Long-Term 용어가 들어가있음)

LSTM(Long Short-Term Memory)
바닐라 RNN의 한계를 극복하기 위해서 LSTM이 등장하게 되었다.
은닉층의 메모리셀에 입력게이트, 망각게이트, 출력게이트를 추가해서 불필요한 기억을 지우고 기억해야할 것들을 정한다. LSTM은 RNN과 비교해서 긴 시퀀스의 입력을 처리하는데 탁월하다. 
이전 시점에서 은닉상태가 다음 시점으로 들어가는 것에 더불어 이전 시점의 "셀 상태"가 등장한다. 셀 상태 또한 다음 시점의 셀 상태 계산할 때 들어가게 된다. 이런 과정을 통해서 LSTM은 RNN과 비교해서 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보인다.

1) 입력 게이트
입력 게이트는 현재 정보를 기억하기 위한 게이트이다. 현재 시점의 입력값과 입력게이트로 이어지는 가중치 1, 이전 시점의 은닉상태가 입력게이트로 이어지는 가중치 2 를 통해서 시그모이드 함수를 지난다 (결과값 1)
또한 현재 시점의 입력값과 입력게이트로 이어지는 가중치 3, 이전 시점의 은닉 상태가 입력게이트로 이어지는 가중치 4을 통해서 하이퍼볼릭 탄젠트 함수를 지난다. (결과값 2)
이 두가지 결과값을 바탕으로 이번에 선택된 기억할 정보의 양을 결정한다.

2) 삭제 게이트
삭제 게이트는 기억을 삭제하기 위한 게이트이다. 현재 시점의 입력값과 이전 시점의 은닉상태가 가중치 5, 6에 의해 연산되고 시그모이드 함수를 지난다.
이 값이 삭제 과정을 거친 정보의 양이다. 0에 가까울수록 정보가 많이 삭제된 것을 의미한다.

3) 셀 상태
입력 게이트에서 구한 두가지 결과값에 대해서 원소별 곱(entrywise product)를 진행한다. 이는 같은 위치의 성분끼리 곱하는 것을 말한다. 이렇게 기억할 값을 결정한다.

입력 게이트에서 선택된 기억을 삭제 게이트의 결과값 이전 시점의 셀상태의 원소별 곱 한 결과와 더한다. 이 값을 현재 시점의 셀 상태라고 한다.

4) 출력 게이트와 은닉상태
출력 게이트는 이전 시점의 은닉상태와 입력값이 가중치들과 곱해지고 시그모이드 함수를 지난 값이다. 또한 이 값을 셀 상태의 결과값과 원소별 곱을 통해서 다음 시점으로 보낼 은닉상태를 만들 수 있다.

게이트 순환 유닛(Gated Recurrent Unit, GRU)
GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서 은닉 상태를 업데이트하는 계산을 줄였다. 복잡했던 LSTM의 구조를 간단하게 만들었다고 보면된다.

케라스에서 SimpleRNN과 LSTM을 사용해보자 !

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN,LSTM, Bidirectional

#임의의 입력을 생성
train_X =  [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))
#단어 벡터의 차원은 5이고 문장의 길이가 4인 경우를 가정한 입력이다.
#즉 4번의 시점이 존재하고 각 시점마다 5차원의 단어벡터가 입력으로 사용된다.
#RNN은 3D텐서를 입력받기 때문에 2D텐서를 3D텐서로 바꿔줘야하고 이는 배치크기를 1로 생각하면 바꿀 수 있다.

(4, 5)


In [5]:
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


In [6]:
#우선 은닉 상태의 크기를 3으로 지정한다.
rnn = SimpleRNN(3)
hidden_state = rnn(train_X)

print('hidden state: {}, shape: {}'.format(hidden_state,hidden_state.shape))

hidden state: [[ 0.07865242 -0.41898796  0.89299065]], shape: (1, 3)


(1,3)크기의 텐서가 출력되는데 이는 마지막 시점의 은닉상태를 나타낸다. return_sequences가 False로 설정되어있기 때문 !


In [8]:
rnn = SimpleRNN(3,return_sequences=True)
hidden_state = rnn(train_X)
print('hidden state: {}, shape: {}'.format(hidden_state,hidden_state.shape))
#return_state가 True일 경우에 return_sequences의 T/F여부와 상관 없이 마지막 시점의 은닉상태를 출력한다.


hidden state: [[[ 0.4791836  -0.6147314   0.99779034]
  [-0.9081396   0.9018332   0.9973853 ]
  [-0.64121014 -0.840876    0.81421214]
  [-0.99414295  0.47461626  0.93201447]]], shape: (1, 4, 3)


In [9]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden states : [[[-0.6792254  -0.94904375 -0.9100806 ]
  [-0.66443324 -0.9978608  -0.9363221 ]
  [-0.11232209 -0.97555333 -0.9650071 ]
  [ 0.8367714  -0.9132827  -0.27311802]]], shape: (1, 4, 3)
last hidden state : [[ 0.8367714  -0.9132827  -0.27311802]], shape: (1, 3)


In [10]:
lstm = LSTM(3,return_sequences=False,return_state=True)
hidden_state,last_state,last_cell_state = lstm(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[-0.16102709 -0.00675711 -0.09994797]], shape: (1, 3)
last hidden state : [[-0.16102709 -0.00675711 -0.09994797]], shape: (1, 3)
last cell state : [[-1.455106   -0.01800066 -0.33717299]], shape: (1, 3)
